<a href="https://colab.research.google.com/github/nicolasrojasv/nicolasrojasv.github.io/blob/main/challenge_8_breakfast_and_lunch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comparing purchases for typical breakfast and lunch items at the most expensive and cheapest supermarkets.



In [ ]:
# Import Required Libraries
import pandas as pd
import numpy as np
import altair as alt

In [ ]:
# Load cleaned prices and items data from Davies and McElvoy (2024)
prices_df = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/pp434/pp434_semi_anonymised_prices.parquet')
items_df = pd.read_parquet('https://autocpi-public.s3.eu-west-2.amazonaws.com/pp434/pp434_semi_anonymised_items.parquet')

In [ ]:
# Merge prices and items on store_id and product_id
df = pd.merge(prices_df, items_df, on=['store_id', 'product_id'], how='inner')
df.sample(5)

,store_id,product_id,date,price,unit_price,loyalty_price,original_price,segment_code,description
669889,2,7999314,2023-08-24,2.25,£1.04 / 100g,NaN,2.25,CP0111402,CEREAL BARS AND CEREAL-BASED SNACKS
1155584,3,910001895629,2024-09-18,4.50,£5.36/kg,NaN,NaN,CP0112501,SAUSAGES AND SIMILAR MEAT PRODUCTS
3430900,8,6614179389893899,2023-12-14,1.75,19.4p/100ml,NaN,2.65,CP0121001,"FRUIT JUICES AND SMOOTHIES, NOT FROM CONCENTRATE"
689424,3,910002827868,2025-03-10,3.60,70.6p/100g,NaN,NaN,CP0111404,OATS AND PORRIDGE
412450,1,304799764,2023-12-25,3.30,10.25 per kg,NaN,NaN,CP0111311,"CAKES, TARTS AND SWEET PIES"


To see which supermarket is the most expensive and which is the cheapest, I use a code provided in class that shows the average and median prices of all supermarkets.

In [ ]:
# Calculate median and mean prices for each store
store_prices = df.copy()
median_prices = store_prices.groupby(['store_id']).agg({'price': ['median', 'mean']}).reset_index()
median_prices.columns = ['store_id', 'median_price', 'mean_price']
median_prices

,store_id,median_price,mean_price
0,1,2.50,4.169827
1,2,2.55,4.205874
2,3,2.25,3.713805
3,4,2.25,3.559485
4,5,1.79,2.598145
5,8,2.50,3.919917
6,9,2.50,3.282282


In [ ]:
# Prepare data for grouped bar chart
median_prices_melted = median_prices.melt(id_vars='store_id', value_vars=['median_price', 'mean_price'], var_name='price_type', value_name='price')
median_prices_melted['store_id'] = "Store " + median_prices_melted['store_id'].astype(str)

# Plot grouped bar chart
alt.Chart(median_prices_melted).mark_bar().encode(
    column=alt.Column('store_id', title=''),
    x=alt.X('price_type', title='', axis=alt.Axis(labels=False)),
    y=alt.Y('price', title='', axis={"labelExpr": "'£' + datum.label", "labelOverlap": False}),
    color='price_type'
).properties(
    title = {
        'text': "Prices by store",
        'subtitle': ["Mean and median prices", ""]
    },
    width=100
)

alt.Chart(...)

The graph above shows that the shop with the cheapest average price is 5, while the most expensive is 1.

In [ ]:
latest_date_store_5 = df[df['store_id'] == 5]['date'].max()
print(f"The last date on which a product from shop 5 appears in the database is: {latest_date_store_5}")

The last date on which a product from shop 5 appears in the database is: 2025-03-25 00:00:00


In [ ]:
first_date_store_5 = df[df['store_id'] == 5]['date'].min()
print(f"The last date on which a product from shop 5 appears in the database is: {first_date_store_5}")

The last date on which a product from shop 5 appears in the database is: 2023-07-28 00:00:00


In [ ]:
latest_date_store_1 = df[df['store_id'] == 1]['date'].max()
print(f"The last date on which a product from shop 1 appears in the database is: {latest_date_store_1}")

The last date on which a product from shop 1 appears in the database is: 2025-11-13 00:00:00


In [ ]:
first_date_store_1 = df[df['store_id'] == 1]['date'].min()
print(f"The last date on which a product from shop 1 appears in the database is: {first_date_store_1}")

The last date on which a product from shop 1 appears in the database is: 2023-07-01 00:00:00


Since store 5 only has information until March 2025, store 4, which has the second lowest prices, is reviewed to see how long it has data.

In [ ]:
latest_date_store_4 = df[df['store_id'] == 4]['date'].max()
print(f"The last date on which a product from shop 4 appears in the database is: {latest_date_store_4}")

The last date on which a product from shop 4 appears in the database is: 2025-11-13 00:00:00


Both shop 1 and shop 4 have data up to 11 November 2025. It is decided to carry out the analysis with these shops.

In [ ]:
first_date_store_4 = df[df['store_id'] == 4]['date'].min()
print(f"The last date on which a product from shop 4 appears in the database is: {first_date_store_4}")

The last date on which a product from shop 4 appears in the database is: 2023-07-10 00:00:00


Store 4 has information from 10 July 2023 onwards, while shop 1 has information from 1 July onwards. The analysis will be carried out from 10 July 2023 onwards.

In [ ]:
# Keep only with store 1 and 4, and from the first date of store 4 onwards
df_filtered = df[df['store_id'].isin([1, 4])]
df_filtered = df_filtered[df_filtered['date'] >= first_date_store_4]

In [ ]:
df_filtered[df_filtered['description'].str.contains('SEMI-SKIMMED MILK', case=False, na=False)]['segment_code'].unique()

array(['CP0114201'], dtype=object)

In [ ]:
df_filtered[df_filtered['description'].str.contains('COFFEE', case=False, na=False)]['segment_code'].unique()

array(['CP0122001'], dtype=object)

In [ ]:
df_filtered[df_filtered['description'].str.contains('BREAD, WHITE', case=False, na=False)]['segment_code'].unique()

array(['CP0111301'], dtype=object)

In [ ]:
df_filtered[df_filtered['description'].str.contains('EGG', case=False, na=False)]['segment_code'].unique()

array(['CP0114801'], dtype=object)

In [ ]:
# Filter for a typical breakfast
df_filtered_breakfast = df_filtered[df_filtered['segment_code'].isin(['CP0114201', 'CP0122001', 'CP0111301', 'CP0114801'])]

In [ ]:
# Obtain the breakfast median price for both store by month
median_prices_breakfast = df_filtered_breakfast.copy()
median_prices_breakfast['year_month'] = median_prices_breakfast['date'].dt.to_period('M')

# Calculate median price for each segment_code, store_id, and year_month
median_by_segment = median_prices_breakfast.groupby(['year_month', 'store_id', 'segment_code'])['price'].median().reset_index()

# Sum these median prices by store_id and year_month
median_prices_by_month_store = median_by_segment.groupby(['year_month', 'store_id'])['price'].sum().reset_index()
median_prices_by_month_store = median_prices_by_month_store.rename(columns={'price': 'sum_median_price'})
display(median_prices_by_month_store.head())

,year_month,store_id,sum_median_price
0,2023-07,1,9.40
1,2023-07,4,9.42
2,2023-08,1,9.40
3,2023-08,4,9.84
4,2023-09,1,9.50


In [ ]:
# Check if all segment codes are present in the entire period for each store
unique_breakfast_segment_codes = median_prices_breakfast.groupby(['year_month', "store_id"])['segment_code'].nunique().reset_index()
display(unique_breakfast_segment_codes.head())
unique_breakfast_segment_codes['year_month'] = unique_breakfast_segment_codes['year_month'].dt.to_timestamp()

alt.Chart(unique_breakfast_segment_codes).mark_line().encode(
    x=alt.X('year_month', title='Month', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('segment_code', title='Number of Unique Segment Codes'),
    color=alt.Color('store_id:N', title='Store', legend=alt.Legend(orient='top', titleOrient='top'))
).properties(
    title='Number of Unique Breakfast Segment Codes by Month'
)

,year_month,store_id,segment_code
0,2023-07,1,4
1,2023-07,4,4
2,2023-08,1,4
3,2023-08,4,4
4,2023-09,1,4


alt.Chart(...)

In [ ]:
#Save the data in a csv
median_prices_by_month_store.to_csv('breakfast_median_prices_by_month_store.csv', index=False)

In [ ]:
# Convert 'year_month' to datetime for plotting
median_prices_by_month_store['year_month'] = median_prices_by_month_store['year_month'].dt.to_timestamp()

alt.Chart(median_prices_by_month_store).mark_line().encode(
    x=alt.X('year_month', title='', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('sum_median_price', title='Median Price', axis=alt.Axis(format='$.2f')),
    color=alt.Color('store_id:N', title='Store', legend=alt.Legend(orient='top', titleOrient='top'))
).properties(
    title='Comparison of Median Price by Month and Store'
)

alt.Chart(...)

Selecting and extracting lunch products

In [ ]:
# Check unique segment code for each product. I selected butter because olive oil was not present in the entire period for one of the stores.
print(df_filtered[df_filtered['description'].str.contains('MEAT OF CHICKEN', case=False, na=False)]['segment_code'].unique())
print(df_filtered[df_filtered['description'].str.contains('RICE', case=False, na=False)]['segment_code'].unique())
print(df_filtered[df_filtered['description'].str.contains('TOMATO', case=False, na=False)]['segment_code'].unique())
print(df_filtered[df_filtered['description'].str.contains('CARROT', case=False, na=False)]['segment_code'].unique())
print(df_filtered[df_filtered['description'].str.contains('BUTTER, DERIVED', case=False, na=False)]['segment_code'].unique())

['CP0112204']
['CP0111101']
['CP0117203']
['CP0117401']
['CP0115201']


In [ ]:
# Filter for a typical lunch
df_filtered_lunch = df_filtered[df_filtered['segment_code'].isin(['CP0112204', 'CP0111101', 'CP0117203', 'CP0117401', 'CP0115201'])]

In [ ]:
# Obtain the lunch median price for both store by month
median_prices_lunch = df_filtered_lunch.copy()
median_prices_lunch['year_month'] = median_prices_lunch['date'].dt.to_period('M')

# Calculate median price for each segment_code, store_id, and year_month
median_by_segment = median_prices_lunch.groupby(['year_month', 'store_id', 'segment_code'])['price'].median().reset_index()

# Sum these median prices by store_id and year_month
median_prices_by_month_store = median_by_segment.groupby(['year_month', 'store_id'])['price'].sum().reset_index()
median_prices_by_month_store = median_prices_by_month_store.rename(columns={'price': 'sum_median_price'})
display(median_prices_by_month_store.head())

,year_month,store_id,sum_median_price
0,2023-07,1,9.840
1,2023-07,4,10.675
2,2023-08,1,10.165
3,2023-08,4,10.880
4,2023-09,1,10.140


In [ ]:
# Check if all segment codes are present in the entire period for each store
unique_lunch_segment_codes = median_prices_lunch.groupby(['year_month', "store_id"])['segment_code'].nunique().reset_index()
display(unique_lunch_segment_codes.head())
unique_lunch_segment_codes['year_month'] = unique_lunch_segment_codes['year_month'].dt.to_timestamp()

alt.Chart(unique_lunch_segment_codes).mark_line().encode(
    x=alt.X('year_month', title='Month', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('segment_code', title='Number of Unique Segment Codes'),
    color=alt.Color('store_id:N', title='Store', legend=alt.Legend(orient='top', titleOrient='top'))
).properties(
    title='Number of Unique Luncha Segment Codes by Month'
)

,year_month,store_id,segment_code
0,2023-07,1,5
1,2023-07,4,5
2,2023-08,1,5
3,2023-08,4,5
4,2023-09,1,5


alt.Chart(...)

In [ ]:
#Save the data in a csv
median_prices_by_month_store.to_csv('lunch_median_prices_by_month_store.csv', index=False)

In [ ]:
# Convert 'year_month' to datetime for plotting
median_prices_by_month_store['year_month'] = median_prices_by_month_store['year_month'].dt.to_timestamp()

alt.Chart(median_prices_by_month_store).mark_line().encode(
    x=alt.X('year_month', title='', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('sum_median_price', title='Median Price', axis=alt.Axis(format='.2f', labelExpr="'£' + datum.label")),
    color=alt.Color('store_id:N', title='Store')
).properties(
    title='Comparison of Median Price by Month and Store'
)

alt.Chart(...)